In [1]:
!pip install yahoofinancials
import yahoofinancials

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

import warnings
warnings.filterwarnings('ignore')

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=0f6ad69bd90d61e27e746b87070f1117e66b0cb2526914c492a07e5f2ab63269
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [4]:
end = '2021-08-03'
start = '2016-07-20'

stock_symbol = 'TCS.NS'

yf = YahooFinancials(stock_symbol)
data = yf.get_historical_price_data(start, end, 'monthly')
TCS = pd.DataFrame(data['TCS.NS']['prices'])

df = TCS.dropna()
df

,date,high,low,open,close,volume,adjclose,formatted_date
0,1469989800,1372.400024,1233.375000,1316.000000,1255.650024,46041780,1139.385742,2016-07-31
1,1472668200,1267.474976,1142.375000,1262.449951,1215.400024,51842742,1102.862549,2016-08-31
2,1475260200,1228.824951,1145.500000,1225.000000,1199.349976,41500766,1088.298828,2016-09-30
3,1477938600,1199.849976,1025.949951,1199.849976,1138.025024,54418020,1035.423462,2016-10-31
4,1480530600,1189.300049,1072.824951,1138.500000,1182.775024,41661230,1076.138672,2016-11-30
...,...,...,...,...,...,...,...,...
56,1617215400,3354.350098,3020.000000,3191.100098,3035.649902,61328558,3014.295166,2021-03-31
57,1619807400,3217.750000,3004.000000,3024.899902,3159.149902,43538924,3136.926025,2021-04-30
58,1622485800,3399.649902,3115.000000,3168.600098,3345.750000,45079239,3338.464355,2021-05-31
59,1625077800,3374.000000,3132.399902,3358.000000,3167.449951,44349890,3160.552490,2021-06-30


In [5]:
df = df[["formatted_date", "close"]]
df.rename(columns={"formatted_date": "ds", "close": "y"}, inplace=True)
df

,ds,y
0,2016-07-31,1255.650024
1,2016-08-31,1215.400024
2,2016-09-30,1199.349976
3,2016-10-31,1138.025024
4,2016-11-30,1182.775024
...,...,...
56,2021-03-31,3035.649902
57,2021-04-30,3159.149902
58,2021-05-31,3345.750000
59,2021-06-30,3167.449951


In [6]:
cut_date = '2021-03-19'
mask1 = (df['ds'] <= cut_date)
mask2 = (df['ds'] > cut_date)

X_tr = df.loc[mask1]
X_tst = df.loc[mask2]
print("train shape",X_tr.shape)
print("test shape",X_tst.shape)

train shape (56, 2)
test shape (5, 2)


In [7]:
X_tst

,ds,y
56,2021-03-31,3035.649902
57,2021-04-30,3159.149902
58,2021-05-31,3345.750000
59,2021-06-30,3167.449951
60,2021-07-31,3560.550049


In [8]:
!pip install neuralprophet

     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 748.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from neuralprophet import NeuralProphet


In [10]:
m = NeuralProphet()
#metrics = m.fit(df, validate_each_epoch=True, valid_p=0.2)
model = m.fit(X_tr, validate_each_epoch=True, freq="B", epochs=100) # epoch=1000


INFO: nprophet.utils - set_auto_seasonalities: Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO: nprophet.utils - set_auto_seasonalities: Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 2


  0%|          | 0/100 [00:00<?, ?it/s]

INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 4.33E-02
Epoch[100/100]: 100%|██████████| 100/100 [00:04<00:00, 21.48it/s, SmoothL1Loss=0.00147, MAE=59.1, RegLoss=0, MAE_val=556, SmoothL1Loss_val=0.128]


In [13]:
future = m.make_future_dataframe(X_tr, periods=5)
forecast = m.predict(future)
forecast.head()

,ds,y,yhat1,residual1,trend,season_yearly
0,2021-03-01,None,1928.431152,NaN,2061.567871,-133.136688
1,2021-03-02,None,1933.296387,NaN,2061.479736,-128.183472
2,2021-03-03,None,1938.530029,NaN,2061.391602,-122.861656
3,2021-03-04,None,1944.093506,NaN,2061.303711,-117.210152
4,2021-03-05,None,1949.946045,NaN,2061.215820,-111.269600


In [14]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(X_tst['y'],forecast['yhat1'])
print("MAPE",round(mape,4))

MAPE 40.2389


In [15]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X_tst['y'],forecast['yhat1'])

1759912.8995507837

Prediction for next 30 days

In [16]:
future = m.make_future_dataframe(df,  periods=30)
forecast = m.predict(future)
forecast

,ds,y,yhat1,residual1,trend,season_yearly
0,2021-08-02,None,2142.001465,NaN,2048.001953,93.999367
1,2021-08-03,None,2137.720215,NaN,2047.913818,89.806374
2,2021-08-04,None,2133.375244,NaN,2047.825806,85.549507
3,2021-08-05,None,2129.004395,NaN,2047.737671,81.266830
4,2021-08-06,None,2124.645752,NaN,2047.649536,76.996216
5,2021-08-09,None,2112.010254,NaN,2047.385254,64.624870
6,2021-08-10,None,2108.061523,NaN,2047.297241,60.764339
7,2021-08-11,None,2104.298340,NaN,2047.209106,57.089310
8,2021-08-12,None,2100.749756,NaN,2047.120972,53.628796
9,2021-08-13,None,2097.441895,NaN,2047.032837,50.409187
